# 第2章：环境搭建与工具准备 - 实战指南

**本章目标**：
- 搭建完整的Python开发环境
- 安装和配置LlamaIndex及相关库
- 准备示例数据集
- 运行第一个RAG程序

**预计时间**：1小时

## 2.1 环境检查

首先检查你的Python版本和基本环境

In [ ]:
import sys
import platform

print("="*60)
print("环境检查报告")
print("="*60)

# Python版本
print(f"\n📌 Python版本: {sys.version}")
print(f"📌 Python路径: {sys.executable}")

# 系统信息
print(f"\n📌 操作系统: {platform.system()} {platform.release()}")
print(f"📌 架构: {platform.machine()}")

# 检查版本是否符合要求
if sys.version_info >= (3, 9):
    print("\n✅ Python版本符合要求 (>= 3.9)")
else:
    print("\n❌ Python版本过低，需要3.9或更高")
    print("   请升级Python: https://www.python.org/downloads/")

print("\n" + "="*60)

## 2.2 安装核心依赖

In [ ]:
# 检查已安装的包
import subprocess
import sys

def check_package(package_name):
    """检查包是否已安装"""
    try:
        __import__(package_name)
        return True
    except ImportError:
        return False

# 核心包列表
packages = {
    "llama_index": "LlamaIndex核心",
    "chromadb": "Chroma向量库",
    "openai": "OpenAI API",
    "pandas": "Pandas数据处理",
    "numpy": "NumPy数值计算"
}

print("\n" + "="*60)
print("核心依赖检查")
print("="*60 + "\n")

for package, description in packages.items():
    if check_package(package):
        print(f"✅ {package:15s} - {description}")
    else:
        print(f"❌ {package:15s} - {description} [未安装]")

print("\n" + "="*60)

# 如果有缺失的包，提示安装命令
missing = [pkg for pkg in packages.keys() if not check_package(pkg)]
if missing:
    print("\n💡 安装命令:")
    print(f"   pip install {' '.join(missing)}")
else:
    print("\n🎉 所有核心依赖已安装！")

## 2.3 配置OpenAI API

In [ ]:
import os
from dotenv import load_dotenv

# 加载.env文件
load_dotenv()

# 检查API密钥
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    # 只显示部分密钥
    masked_key = api_key[:8] + "..." + api_key[-4:]
    print(f"✅ OpenAI API密钥已配置: {masked_key}")
else:
    print("❌ 未检测到OPENAI_API_KEY")
    print("\n配置步骤:")
    print("1. 创建.env文件")
    print("2. 添加: OPENAI_API_KEY=your_key_here")
    print("3. 重新加载此notebook")

## 2.4 准备示例数据

In [ ]:
# 创建示例数据目录
import os
from pathlib import Path

# 创建目录结构
directories = [
    "data/raw",
    "data/processed",
    "data/eval"
]

for directory in directories:
    Path(directory).mkdir(parents=True, exist_ok=True)
    print(f"✅ 创建目录: {directory}")

print("\n目录结构创建完成！")

In [ ]:
# 创建示例文档
sample_docs = {
    "data/processed/python_intro.txt": """Python是一种高级编程语言

Python由Guido van Rossum于1991年创建。它是一种解释型语言，
具有动态类型和自动内存管理功能。

Python的设计哲学强调代码的可读性，使用缩进来定义代码块。
这使得Python代码易于阅读和维护。
""",
    
    "data/processed/python_features.txt": """Python的特点

Python有许多吸引人的特点：

1. 语法简洁：Python的语法清晰简洁，容易学习
2. 丰富的库：标准库和第三方库非常丰富
3. 跨平台：可以在Windows、Mac、Linux上运行
4. 多范式：支持面向对象、函数式等多种编程范式
5. 应用广泛：Web开发、数据分析、人工智能等
""",
    
    "data/processed/rag_intro.txt": """RAG技术概述

RAG（Retrieval-Augmented Generation）是一种结合了信息检索
和文本生成的AI技术。

RAG的核心步骤包括：
1. 检索：从知识库中查找相关文档
2. 增强：将检索到的文档加入提示词
3. 生成：LLM基于增强的提示词生成答案
"""
}

# 保存文件
for filepath, content in sample_docs.items():
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(content.strip())
    print(f"✅ 创建文件: {filepath}")

print("\n示例文档创建完成！")

## 2.5 测试环境

运行一个简单的测试，确保所有组件工作正常

In [ ]:
print("="*60)
print("测试环境配置")
print("="*60)

# 测试1: 导入核心库
print("\n测试1: 导入核心库")
try:
    from llama_index.core import SimpleDirectoryReader, Document
    from llama_index.embeddings.openai import OpenAIEmbedding
    from llama_index.llms.openai import OpenAI as OpenAILLM
    import chromadb
    print("✅ 所有核心库导入成功")
except Exception as e:
    print(f"❌ 导入失败: {e}")

# 测试2: 加载文档
print("\n测试2: 加载文档")
try:
    reader = SimpleDirectoryReader("data/processed")
    docs = reader.load_data()
    print(f"✅ 成功加载 {len(docs)} 个文档")
    for i, doc in enumerate(docs, 1):
        print(f"   文档{i}: {len(doc.text)} 字符")
except Exception as e:
    print(f"❌ 加载失败: {e}")

# 测试3: OpenAI连接（如果有API密钥）
if api_key:
    print("\n测试3: OpenAI API连接")
    try:
        embed_model = OpenAIEmbedding(model="text-embedding-3-small")
        # 测试生成嵌入
        test_embedding = embed_model.get_text_embedding("测试")
        print(f"✅ API连接正常，嵌入维度: {len(test_embedding)}")
    except Exception as e:
        print(f"❌ API连接失败: {e}")
else:
    print("\n跳过API测试（未配置密钥）")

print("\n" + "="*60)
print("环境测试完成！")
print("="*60)

## 2.6 快速体验：第一个RAG程序

In [ ]:
# 如果配置了API密钥，运行这个简单的RAG示例

if api_key:
    from llama_index.core import VectorStoreIndex, StorageContext
    from llama_index.vector_stores.chroma import ChromaVectorStore
    import chromadb
    
    print("\n" + "="*60)
    print("快速体验：构建第一个RAG系统")
    print("="*60 + "\n")
    
    # 1. 加载文档
    print("步骤1: 加载文档...")
    reader = SimpleDirectoryReader("data/processed")
    documents = reader.load_data()
    print(f"✅ 加载了 {len(documents)} 个文档")
    
    # 2. 创建向量索引
    print("\n步骤2: 创建向量索引...")
    chroma_client = chromadb.PersistentClient(path="./test_chroma_db")
    collection = chroma_client.get_or_create_collection("test_collection")
    vector_store = ChromaVectorStore(chroma_collection=collection)
    
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        show_progress=True
    )
    print("✅ 索引创建完成")
    
    # 3. 测试查询
    print("\n步骤3: 测试查询...")
    query_engine = index.as_query_engine(
        llm=OpenAILLM(model="gpt-3.5-turbo", temperature=0.7),
        similarity_top_k=2
    )
    
    test_questions = [
        "Python是什么时候创建的？",
        "Python有哪些特点？",
        "什么是RAG？"
    ]
    
    for question in test_questions:
        print(f"\n❓ 问题: {question}")
        response = query_engine.query(question)
        print(f"\n📝 答案: {response}")
        print("-" * 60)
    
    print("\n" + "="*60)
    print("🎉 恭喜！你的第一个RAG系统运行成功！")
    print("="*60)

else:
    print("\n⚠️ 要运行此示例，请先配置OPENAI_API_KEY")
    print("\n配置方法:")
    print("1. 创建.env文件")
    print("2. 添加: OPENAI_API_KEY=sk-your-key-here")
    print("3. 重启Jupyter")

## 2.7 环境检查清单

完成以下检查项，确保环境配置正确：

- [ ] Python 3.10+已安装
- [ ] 虚拟环境已创建
- [ ] LlamaIndex已安装
- [ ] Chroma已安装
- [ ] OpenAI API密钥已配置
- [ ] 示例数据已准备
- [ ] 测试程序运行成功

### 下一步

✅ 环境配置完成后，进入第3章：基础RAG实现

### 遇到问题？

查看常见问题：[FAQ](../exercises/module1_exercises.md)